In [174]:
 
import urllib.request
import pandas as pd
import nltk
from nltk.tokenize import RegexpTokenizer

stories = {
    "pride_and_predjudice": {
        "url": "https://www.gutenberg.org/cache/epub/1342/pg1342.txt",
        "start_word_pos": 6196,
        "end_word_pos": 154559
        }
}

def get_story(story_key):
    return stories[story_key]

def get_word_frequency(words):
    """
    Generates word frequency data including counts and probabilities.
    
    Parameters:
    - words (list): List of words to analyze for frequency.
    
    Returns:
    - DataFrame: DataFrame with word counts.
    """
    bow = pd.Series(words).value_counts().reset_index()
    bow.columns = ['word', 'counts']        
    bow = bow.sort_values(by='counts', ascending=False).reset_index(drop=True)
    print(bow)
    return bow


In [175]:
pride_and_prejudice = get_story("pride_and_predjudice")
url = pride_and_prejudice["url"]
word_start = pride_and_prejudice["start_word_pos"]
word_end = pride_and_prejudice["end_word_pos"]

response = urllib.request.urlopen(url)
content = response.read().decode('utf-8')
tokenizer = RegexpTokenizer(r'\w+|[^\w\s]')
words = tokenizer.tokenize(content) 
words = words[word_start:word_end]
# print first 10 words
print(words[:10])
# print last 10 words
print(words[-10:])
word_freq_df = get_word_frequency(words)


['It', 'is', 'a', 'truth', 'universally', 'acknowledged', ',', 'that', 'a', 'single']
['Derbyshire', ',', 'had', 'been', 'the', 'means', 'of', 'uniting', 'them', '.']
                word  counts
0                  ,    9497
1                  .    6251
2                 to    4130
3                the    4077
4                 of    3619
...              ...     ...
6854             Any       1
6855  congratulatory       1
6856     ingratitude       1
6857      revolution       1
6858      reluctance       1

[6859 rows x 2 columns]
